GAN
===

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # 상태바 라이브러리

from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout # Dropout : 인공 신경망의 뉴런을 확률적으로 사용하지 않음으로써 과적합을 방지하는 기법 
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import initializers

In [2]:
# keras가 tensorflow를 백엔드로 사용할 수 있도록 설정한다
os.environ['KERAS_BACKEND'] = 'tensorflow'

# 실험을 재현하고 동일한 결과를 얻을 수 있는지 확인하기 위해 seed 설정
np.random.seed(10)

# 우리의 랜덤 노이즈 벡터의 차원을 설정
random_dim = 100

In [12]:
def load_mnist_data():
    # load data
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    
    # normalize
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    
    # 한 개의 row 당 784개의 columns를 가지게 shape을 바꿔줌
    X_train = X_train.reshape(60000, 784)
    return (X_train, y_train, X_test, y_test)

In [4]:
# Adam optimizer 사용
def get_optimizer():
    return Adam(lr = 0.0002, beta_1=0.5)

# generator
def get_generator(optimizer):
    generator = Sequential()
    generator.add(Dense(256, # output의 수
                        input_dim=random_dim, 
                       kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(512))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(784, activation='tanh'))
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)
    return generator

# discriminator
def get_discriminator(optimizer):
    discriminator = Sequential()
    discriminator.add(Dense(1024, 
                            input_dim=784,
                           kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3)) # input unit을 0.3만큼 사용함
    
    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(Dense(1, activation='sigmoid'))
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
    return discriminator

In [17]:
def get_gan_network(discriminator, random_dim, generator, optimizer):
    # generator와 discriminator을 동시에 학습시키고 싶을 때 trainable을 False로 설정
    discriminator.trainable = False
    
    # GAN 입력(노이즈)은 100차원으로 설정
    gan_input = Input(shape=(random_dim))
    
    # generator의 결과는 이미지이다
    X = generator(gan_input)
    
    # Discriminator의 결과는 이미지가 진짜인지 가짜인지에 대한 확률이다
    gan_output = discriminator(X)
    
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer)
    return gan

In [6]:
# 생성된 MNIST 이미지를 보여주는 함수
def plot_generated_images(epoch, generator, examples=100, dim=(10,10), figsize=(10,10)):
    noise = np.random.normal(0, 1, size=[examples, random_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 28, 28)
    
    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' %epoch)

In [ ]:
def get_gan_network(discriminator, random_dim, generator, optimizer):
    # 우리는 Generator와 Discriminator를 동시에 학습시키고 싶을 때 trainable을 False로 설정합니다.
    discriminator.trainable = False

    # GAN 입력 (노이즈)은 위에서 100 차원으로 설정
    gan_input = Input(shape=(random_dim,))

    # Generator의 결과는 이미지 입니다.
    x = generator(gan_input)

    # Discriminator의 결과는 이미지가 진짜인지 가짜인지에 대한 확률
    gan_output = discriminator(x)

    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer)
    return gan

 
# 생성된 MNIST 이미지를 보여주는 함수
def plot_generated_images(epoch, generator, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, random_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)

    
def train(epochs=1, batch_size=128):
    x_train, y_train, x_test, y_test = load_mnist_data()

    # train 데이터를 128 사이즈의 batch 로 나눔
    batch_count = x_train.shape[0] // batch_size

    # 우리의 GAN 네트워크를 듦
    adam = get_optimizer()
    generator = get_generator(adam)
    discriminator = get_discriminator(adam)
    gan = get_gan_network(discriminator, random_dim, generator, adam)

    for e in range(1, epochs+1):
        print('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in tqdm(range(batch_count)):
            # 입력으로 사용할 random 노이즈와 이미지를 가져옴
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])

            image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]

            # MNIST 이미지를 생성
            generated_images = generator.predict(noise)
            X = np.concatenate([image_batch, generated_images])

            y_dis = np.zeros(2*batch_size)
            y_dis[:batch_size] = 0.9

            # Discriminator를 학습
            discriminator.trainable = True
            discriminator.train_on_batch(X, y_dis)

            # Generator를 학습
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y_gen)

        if e == 1 or e % 20 == 0:
            plot_generated_images(e, generator)

if __name__ == '__main__':
    train(50, 128)